In [ ]:
import pickle 
import numpy as np
from gensim.models.word2vec import Word2Vec
from gensim.corpora.dictionary import Dictionary
import jieba
import jieba.analyse

我们有用户评论数据分为正负样本，通过lstm训练来得到一个模型，通过该型我们预测一句评论是正样本或者是负样本
1.将所有评论汇总
2.通过结巴分词和word2vec生成词嵌量
3.使用keras进行训练返回模型
4.测试

In [3]:
# sentences_list = Textsta('/home/zer0/桌面/用户评论情感分析/中文情感分析语料库/clothing/neg.txt')
# 这个只需运行一次
# 读取所有的语料文本，写入一个总文件同时生成标签
# 以neg.txt 结尾负样本,以pos.txt结尾正样本
y=[]
def create_all(filename,y):
    with open('all.txt','a+') as t:
        f = open('中文情感分析语料库'+filename,'r',encoding='gbk',errors='ignore')
        data = f.read()
        t.write(data+'\n')
        count = open('中文情感分析语料库'+filename,'r',encoding='gbk',errors='ignore').readlines()
        for i in range(len(count)):
#             print(filename[-7:])
            if filename[-7:]=='neg.txt':
                y.append(0)
            else:
                y.append(1)
        f.close()
        t.close()

file_list = ['/clothing/neg.txt','/clothing/pos.txt','/fruit/neg.txt','/fruit/pos.txt','/hotel/neg.txt','/hotel/pos.txt','/pda/neg.txt','/pda/pos.txt','/shampoo/neg.txt','/shampoo/pos.txt']

for i in file_list:
    create_all(i,y)

In [2]:

# 传进一个文件名，返回该文件分词后的列表

def Textsta(filename): 
    f = open(filename,'r',encoding='utf-8')
    sentences_list = []
    for line in f:
        single_list = line.strip().split(' ')
        single_list = jieba.analyse.extract_tags(single_list[0],topK=20,withWeight=False,allowPOS=())
        while '' in single_list:
            single_list.remove('')
        sentences_list.append(single_list)
    f.close()
    return sentences_list


In [4]:
# 调用word2vec 模型返回字典和词的向量

def create_dictionaries(model):
    gensim_dict = Dictionary()
    gensim_dict.doc2bow(model.wv.vocab.keys(),allow_update=True)
    w2indx = {v:k+1 for k,v in gensim_dict.items()}
    w2vec = {word:model[word] for word in w2indx.keys()}
    return w2indx,w2vec

In [5]:
#对all.txt 分词
sentence = Textsta('all.txt')
# word2vec 方法转向量
model = Word2Vec(sentence,size=100,min_count=5,window=5)
index_dict,word_vectors = create_dictionaries(model)
# print(index_dict)
print(word_vectors['熟练'])
# 可以看到这个词在字典中的表示就是这样的，我们也可以保存到本地
# with open('word_vec.txt','w') as f:
#     f.write(word_vectors)

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.756 seconds.
Prefix dict has been built succesfully.


[ 0.05399641 -0.10912966  0.12273714 -0.04977618  0.08095848  0.0123782
 -0.04039728 -0.07805733 -0.04894287  0.11321828  0.06093043  0.1166272
  0.00669917 -0.00227427 -0.01535578 -0.01468829  0.04204039 -0.07025021
  0.16036604  0.00890837 -0.03197939 -0.10754133  0.06049434  0.08962453
 -0.0033539   0.08726767  0.0532543  -0.01957593  0.04428588  0.0558311
  0.10793772  0.0305222   0.04336311 -0.01237232  0.03882742 -0.09393188
  0.0361244  -0.08009711  0.09666263  0.05550296 -0.01417902  0.04177978
  0.13866548  0.02448069  0.0995876   0.07853965  0.05106522 -0.03618902
 -0.05889981  0.16454238  0.14964625 -0.10103108  0.00411489 -0.13777688
 -0.06397779  0.0111174  -0.02855334 -0.10617605 -0.01923109  0.03118331
 -0.02526609  0.01191539 -0.01318542 -0.01041531  0.01357579 -0.02841787
 -0.06332406  0.06217002 -0.06881575  0.05133618  0.03116947  0.01810011
  0.00404764  0.12125523 -0.04075477 -0.05919284 -0.13271192  0.03628181
  0.10039835  0.06864693  0.03264365 -0.01654016 -0.01

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  import sys


In [6]:
# 生成词嵌入向量
# 把上边一个词向量的长度转为100
n_symbols = len(index_dict) +1
embedding_weights = np.zeros((n_symbols,100))
for w,index in index_dict.items():
    embedding_weights[index,:] = word_vectors[w]
print(embedding_weights.shape)
# 词有7767个

(7767, 100)


In [7]:
# 在字典中找到词返回索引
# 文本和词典匹配将我们的词特征转为数字
def text_to_index_array(dic,sentence):
    new_sentence = []
    for sen in sentence:
        new_sen = []
        for word in sen:
            try:
                new_sen.append(dic[word])
            except:
                new_sen.append(0)
        new_sentence.append(new_sen)
    return np.array(new_sentence)

x = text_to_index_array(index_dict,sentence)
# print(x[10])

In [8]:
import numpy as np
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM
from keras.layers.core import Dense,Dropout,Activation
from sklearn.cross_validation import train_test_split


Using TensorFlow backend.
/usr/local/lib/python3.5/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [9]:
# 划分数据集生成训练和测试4:1
# sklearn包划分数据
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2)
# 把x输入特征标准化，不够的补0是每一个输入的x 为50长度
x_train = sequence.pad_sequences(x_train,maxlen=50)
x_test = sequence.pad_sequences(x_test,maxlen=50)
x_train = np.array(x_train)
x_test = np.array(x_test)
y_train = np.array(y_train)
y_test = np.array(y_test)
print(x_train.shape)

(40000, 50)


In [10]:
model = Sequential()
model.add(Embedding(output_dim=100,input_dim=n_symbols,mask_zero=True,weights=[embedding_weights]))
model.add(LSTM(output_dim=50,activation='sigmoid',inner_activation='hard_sigmoid'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:3: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(recurrent_activation="hard_sigmoid", units=50, activation="sigmoid")`
  This is separate from the ipykernel package so we can avoid doing imports until


In [11]:
model.fit(x_train,y_train,batch_size=64,epochs=10,validation_data=(x_test,y_test))
score,acc = model.evaluate(x_test,y_test,batch_size=32)
print(score,acc)
print(model.summary())

Train on 40000 samples, validate on 10000 samples
Epoch 1/10
40000/40000 [==============================] - 44s 1ms/step - loss: 0.4036 - acc: 0.8091 - val_loss: 0.2648 - val_acc: 0.8954
Epoch 2/10
40000/40000 [==============================] - 44s 1ms/step - loss: 0.2461 - acc: 0.9041 - val_loss: 0.2591 - val_acc: 0.8999
Epoch 3/10
40000/40000 [==============================] - 46s 1ms/step - loss: 0.2140 - acc: 0.9186 - val_loss: 0.2556 - val_acc: 0.9016
Epoch 4/10
40000/40000 [==============================] - 45s 1ms/step - loss: 0.1954 - acc: 0.9271 - val_loss: 0.2656 - val_acc: 0.9013
Epoch 5/10
40000/40000 [==============================] - 45s 1ms/step - loss: 0.1818 - acc: 0.9328 - val_loss: 0.2767 - val_acc: 0.8981
Epoch 6/10
40000/40000 [==============================] - 45s 1ms/step - loss: 0.1707 - acc: 0.9385 - val_loss: 0.2958 - val_acc: 0.8944
Epoch 7/10
40000/40000 [==============================] - 45s 1ms/step - loss: 0.1625 - acc: 0.9413 - val_loss: 0.3093 - val_acc

In [13]:
# 直接输入一句话对其进行预测
def convert_vector_predict(str_r):
    new_str = jieba.analyse.extract_tags(str_r,topK=20,withWeight=False,allowPOS=())
#     print(new_str)
    x = text_to_index_array(index_dict,[new_str])
    x = sequence.pad_sequences(x,maxlen=50)
#     print(x)
    y = model.predict_classes(x)
    return y
print(convert_vector_predict('好可爱啊'))

[[1]]
